In [1]:
# Functions taken from various internet sources.

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import FeatureHasher

In [6]:
df_flights = pd.read_csv('https://raw.githubusercontent.com/ismayc/pnwflights14/master/data/flights.csv')
df_flights.head(3)
df_flights = df_flights.dropna()

df_flights.reindex(range(1, len(df_flights)+1))

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
1,2014.0,1.0,1.0,4.0,-6.0,738.0,-23.0,US,N195UW,1830.0,SEA,CLT,252.0,2279.0,0.0,4.0
2,2014.0,1.0,1.0,8.0,13.0,548.0,-4.0,UA,N37422,1609.0,PDX,IAH,201.0,1825.0,0.0,8.0
3,2014.0,1.0,1.0,28.0,-2.0,800.0,-23.0,US,N547UW,466.0,PDX,CLT,251.0,2282.0,0.0,28.0
4,2014.0,1.0,1.0,34.0,44.0,325.0,43.0,AS,N762AS,121.0,SEA,ANC,201.0,1448.0,0.0,34.0
5,2014.0,1.0,1.0,37.0,82.0,747.0,88.0,DL,N806DN,1823.0,SEA,DTW,224.0,1927.0,0.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160744,2014.0,9.0,28.0,655.0,-5.0,831.0,-14.0,VX,N842VA,84.0,PDX,SFO,81.0,550.0,6.0,55.0
160745,2014.0,9.0,28.0,656.0,-4.0,1031.0,-3.0,AS,N523AS,676.0,SEA,DEN,142.0,1024.0,6.0,56.0
160746,2014.0,9.0,28.0,657.0,-3.0,931.0,-3.0,AS,N625AS,466.0,SEA,LAX,131.0,954.0,6.0,57.0
160747,2014.0,9.0,28.0,657.0,-3.0,1209.0,-10.0,DL,N582NW,2314.0,SEA,MSP,168.0,1399.0,6.0,57.0


In [19]:
dff = df_flights
dff2 = df_flights

In [9]:
plot_cat = pd.DataFrame(index = range(len(dff)))
for cls in dff.columns:
    if dff[cls].dtype == 'object':
        plot_cat = plot_cat.join(dff[cls])
       
    
# plot_cat.apply(pd.value_counts).plot(kind="barh", subplots=True, layout=(2,2), legend=False)
# plt.tight_layout()  
# plt.show()

#         counter = dff[cls].value_counts()
#         sns.barplot(counter.index, counter.values, alpha=0.9)
#         plt.show()

In [11]:
cat_col = ['carrier', 'tailnum', 'origin', 'dest']

In [10]:
map_dict = {}
for cls in plot_cat.columns:
    tmp1 = list(dff[cls].unique())
    tmp2 = list(range(len(tmp1)))
    
    map_dict[cls] = dict(zip(tmp2, tmp1))

##### MCFS (Multi-Cluster Feature Selection)

In [4]:
# uncompyle6 version 3.5.0
# Python bytecode 2.7 (62211)
# Decompiled from: Python 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
# Embedded file name: MCFS.pyc
# Compiled at: 2016-06-01 02:35:54
import scipy
from sklearn import linear_model
from skfeature.utility.construct_W import construct_W

def mcfs(X, n_selected_features, **kwargs):
    """
    This function implements unsupervised feature selection for multi-cluster data.

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    n_selected_features: {int}
        number of features to select
    kwargs: {dictionary}
        W: {sparse matrix}, shape (n_samples, n_samples)
            affinity matrix
        n_clusters: {int}
            number of clusters (default is 5)

    Output
    ------
    W: {numpy array}, shape(n_features, n_clusters)
        feature weight matrix

    Reference
    ---------
    Cai, Deng et al. "Unsupervised Feature Selection for Multi-Cluster Data." KDD 2010.
    """
    if 'W' not in kwargs:
        W = construct_W(X)
    else:
        W = kwargs['W']
    if 'n_clusters' not in kwargs:
        n_clusters = 5
    else:
        n_clusters = kwargs['n_clusters']
    W = W.toarray()
    W = (W + W.T) / 2
    W_norm = np.diag(np.sqrt(1 / W.sum(1)))
    W = np.dot(W_norm, np.dot(W, W_norm))
    WT = W.T
    W[W < WT] = WT[(W < WT)]
    eigen_value, ul = scipy.linalg.eigh(a=W)
    Y = np.dot(W_norm, ul[:, -1 * n_clusters - 1:-1])
    n_sample, n_feature = X.shape
    W = np.zeros((n_feature, n_clusters))
    for i in range(n_clusters):
        clf = linear_model.Lars(n_nonzero_coefs=n_selected_features)
        clf.fit(X, Y[:, i])
        W[:, i] = clf.coef_

    return W


def feature_ranking(W):
    """
    This function computes MCFS score and ranking features according to feature weights matrix W
    """
    mcfs_score = W.max(1)
    idx = np.argsort(mcfs_score, 0)
    idx = idx[::-1]
    return idx

In [42]:
dff_mcfs = np.array(dff_dc[0:5000])

In [72]:
W1 = mcfs(dff_mcfs, n_selected_features = 10, n_clusters = 10)

C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:578: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 6 iterations, i.e. alpha=7.409e-06, with an active set of 6 regressors, and the smallest cholesky pivot element being 1.054e-08. Reduce max_iter or increase eps parameters.
  ConvergenceWarning)


In [74]:
feature_ranking(W1)

array([17, 72, 62, 36, 46, 64, 30, 90, 41, 38, 79, 87, 68, 40, 84, 56, 24,
       61, 43, 23, 25, 10,  5,  0, 12, 69, 88, 19, 20, 21, 22, 86, 85, 83,
       26, 27, 28, 29, 82, 31, 32, 18, 16, 70, 15, 14, 13, 11, 89,  9,  8,
        7,  6, 91,  4,  3,  2,  1, 33, 34, 35, 60, 54, 55, 75, 57, 58, 59,
       74, 81, 73, 63, 71, 65, 66, 67, 53, 52, 51, 50, 49, 48, 47, 92, 45,
       44, 76, 42, 77, 78, 39, 80, 37, 93], dtype=int64)

In [62]:
feature_ranking(W) # n_sel_feat = 5, n_clust = 5

array([72, 90, 38, 68, 87, 40, 31, 23, 24, 25, 26, 27, 28, 29, 30, 93, 32,
       22, 34, 35, 36, 37, 39, 41, 42, 33, 20, 21, 44,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 43, 46, 45,
       92, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       88, 89, 91, 69, 67, 66, 55, 47, 48, 49, 50, 51, 52, 53, 54, 56, 65,
       57, 58, 59, 60, 61, 62, 63, 64,  0], dtype=int64)

In [75]:
mat = scipy.io.loadmat('COIL20.mat')

In [86]:
from skfeature.utility import unsupervised_evaluation

X = mat['fea']    # data
X = X.astype(float)
y = mat['gnd']    # label
y = y[:, 0]

kwargs = {"metric": "euclidean", "neighborMode": "knn", "weightMode": "heatKernel", "k": 5, 't': 1}
W = construct_W(X, **kwargs)

num_fea = 100    # specify the number of selected features
num_cluster = 20    # specify the number of clusters, it is usually set as the number of classes in the ground truth

# obtain the feature weight matrix
Weight = mcfs(X, n_selected_features=num_fea, W=W, n_clusters=20)

# sort the feature scores in an ascending order according to the feature scores
idx = feature_ranking(Weight)

# obtain the dataset on the selected features
selected_features = X[:, idx[0:num_fea]]

# perform kmeans clustering based on the selected features and repeats 20 times
nmi_total = 0
acc_total = 0
for i in range(0, 20):
    nmi, acc = unsupervised_evaluation.evaluation(X_selected=selected_features, n_clusters=num_cluster, y=y)
    nmi_total += nmi
    acc_total += acc

# output the average NMI and average ACC
print('NMI:', float(nmi_total)/20)
print('ACC:', float(acc_total)/20)

C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average

C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.l

NMI: 0.7312533096875604
ACC: 0.6447569444444444


C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\utils\linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


##### Label Encoding

In [15]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [25]:
le = LabelEncoder()
labeller = []
for keys in map_dict.keys():
    g_labels = le.fit_transform(dff[keys])
    labeller.append(str(keys+'_label'))
    dff[labeller[-1]] = g_labels

In [26]:
labeller2 = labeller
labeller2.remove('tailnum_label')

##### One-Hot Encoding

In [247]:
dff_ohe = dff
dff_ohe.reset_index(drop=True, inplace=True)
ohe = OneHotEncoder()

for labels in labeller:
    if labels != 'tailnum_label':
        g_labels = le.fit_transform(dff[labels.split('_')[0]])
        tmp1 = ohe.fit_transform(dff[[labels]]).toarray()
        tmp2 = list(le.classes_)
        tmp3 = pd.DataFrame(tmp1, columns = tmp2)
        tmp3.reset_index(drop=True, inplace=True)
        tmp4 = pd.concat([dff_ohe, tmp3], axis = 1, ignore_index = True)
        dff_ohe = tmp4
        print(dff_ohe.shape)
#         dff_ohe = pd.merge(dff_ohe, tmp3, on=['placeholder_to_merge'])

C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(160748, 32)


C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(160748, 34)


C:\Users\Intern\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(160748, 105)


In [220]:
# Remove columns that has been encoded
dff_ohe = dff_ohe.drop(columns = cat_col)
dff_ohe = dff_ohe.drop(columns = labeller2)

In [221]:
dff_ohe

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,flight,air_time,distance,...,SEA_y,SFO,SIT,SJC,SLC,SMF,SNA,STL,TPA,TUS
0,2014,1,1,1.0,96.0,235.0,70.0,145,194.0,1542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2014,1,1,4.0,-6.0,738.0,-23.0,1830,252.0,2279,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2014,1,1,8.0,13.0,548.0,-4.0,1609,201.0,1825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2014,1,1,28.0,-2.0,800.0,-23.0,466,251.0,2282,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2014,1,1,34.0,44.0,325.0,43.0,121,201.0,1448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160743,2014,9,28,655.0,40.0,1613.0,115.0,616,345.0,2378,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160744,2014,9,28,655.0,-5.0,831.0,-14.0,84,81.0,550,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160745,2014,9,28,656.0,-4.0,1031.0,-3.0,676,142.0,1024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160746,2014,9,28,657.0,-3.0,931.0,-3.0,466,131.0,954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Dummy Coding

In [37]:
dff_dc = dff
for label in cat_col:
    if label != 'tailnum':
        tmp1 = pd.get_dummies(dff[label], drop_first = True)
        dff_dc = pd.merge(dff_dc, tmp1, left_index = True, right_index = True)
    
dff_dc = dff_dc.drop(columns = labeller2)
dff_dc = dff_dc.drop(columns = cat_col)

In [38]:
dff_dc

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,flight,air_time,distance,...,SEA_y,SFO,SIT,SJC,SLC,SMF,SNA,STL,TPA,TUS
0,2014,1,1,1.0,96.0,235.0,70.0,145,194.0,1542,...,0,0,0,0,0,0,0,0,0,0
1,2014,1,1,4.0,-6.0,738.0,-23.0,1830,252.0,2279,...,0,0,0,0,0,0,0,0,0,0
2,2014,1,1,8.0,13.0,548.0,-4.0,1609,201.0,1825,...,0,0,0,0,0,0,0,0,0,0
3,2014,1,1,28.0,-2.0,800.0,-23.0,466,251.0,2282,...,0,0,0,0,0,0,0,0,0,0
4,2014,1,1,34.0,44.0,325.0,43.0,121,201.0,1448,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160743,2014,9,30,2344.0,-1.0,527.0,-5.0,1215,203.0,1721,...,0,0,0,0,0,0,0,0,0,0
160744,2014,9,30,2347.0,-3.0,229.0,-2.0,121,209.0,1448,...,0,0,0,0,0,0,0,0,0,0
160745,2014,9,30,2349.0,-1.0,514.0,-19.0,660,188.0,1660,...,0,0,0,0,0,0,0,0,0,0
160746,2014,9,30,2357.0,-2.0,545.0,-20.0,479,210.0,1825,...,0,0,0,0,0,0,0,0,0,0


##### Feature-hashing

In [31]:
from sklearn.feature_extraction import FeatureHasher

In [32]:
dff_fh = dff
dff_fh.reset_index(drop=True, inplace=True)

for label in cat_col:
    if label != 'tailnum':
        n_ft = int(np.floor(len(dff_fh[label].unique())/2)+1)
        fh = FeatureHasher(n_features = n_ft, input_type = 'string')
        
        hashed_features = fh.fit_transform(dff_fh[label])
        hashed_features = hashed_features.toarray()
        clnm = [label + "_" + str(num) for num in range(n_ft)]
        tmp1 = pd.DataFrame(hashed_features, columns = clnm)
        tmp1.reset_index(drop=True, inplace=True)
        tmp2 = pd.concat([dff_fh, tmp1], axis = 1)
        dff_fh = tmp2
        
dff_fh = dff_fh.drop(columns = labeller2)
dff_fh = dff_fh.drop(columns = cat_col)

In [34]:
dff_fh.columns

Index(['year', 'month', 'day', 'dep_time', 'dep_delay', 'arr_time',
       'arr_delay', 'flight', 'air_time', 'distance', 'hour', 'minute',
       'tailnum_label', 'carrier_0', 'carrier_1', 'carrier_2', 'carrier_3',
       'carrier_4', 'carrier_5', 'origin_0', 'origin_1', 'dest_0', 'dest_1',
       'dest_2', 'dest_3', 'dest_4', 'dest_5', 'dest_6', 'dest_7', 'dest_8',
       'dest_9', 'dest_10', 'dest_11', 'dest_12', 'dest_13', 'dest_14',
       'dest_15', 'dest_16', 'dest_17', 'dest_18', 'dest_19', 'dest_20',
       'dest_21', 'dest_22', 'dest_23', 'dest_24', 'dest_25', 'dest_26',
       'dest_27', 'dest_28', 'dest_29', 'dest_30', 'dest_31', 'dest_32',
       'dest_33', 'dest_34', 'dest_35'],
      dtype='object')

##### Laplacian Score + Feature-based Entropy

In [274]:
from skfeature.utility.construct_W import construct_W

def LaplacianScore(X, neighbour_size = 5,  t = 1):
    W = construct_W(X,t=t,neighbour_size=neighbour_size)
    n_samples, n_features = np.shape(X)
    
    #construct the diagonal matrix
    D=np.array(W.sum(axis=1))
    D = scipy.sparse.diags(np.transpose(D), [0])
    #construct graph Laplacian L
    L=D-W.toarray()

    #construct 1= [1,···,1]' 
    I=np.ones((n_samples,n_features))

    #construct fr' => fr= [fr1,...,frn]'
    Xt = np.transpose(X)

    #construct fr^=fr-(frt D I/It D I)I
    t=np.matmul(np.matmul(Xt,D.toarray()),I)/np.matmul(np.matmul(np.transpose(I),D.toarray()),I)
    t=t[:,0]
    t=np.tile(t,(n_samples,1))
    fr=X-t

    #Compute Laplacian Score
    fr_t=np.transpose(fr)
    Lr=np.matmul(np.matmul(fr_t,L),fr)/np.matmul(np.dot(fr_t,D.toarray()),fr)

    return np.diag(Lr)

def distanceEntropy(d, mu = 0.5, beta=10):
    """
    As per: An Unsupervised Feature Selection Algorithm: Laplacian Score Combined with
    Distance-based Entropy Measure, Rongye Liu 
    """
    if d<=mu:
        result = (np.exp(beta * d) - np.exp(0))/(np.exp(beta * mu) - np.exp(0))
    else:
        result = (np.exp(beta * (1-d) )- np.exp(0))/(np.exp(beta *(1- mu)) - np.exp(0))              
    return result


def lse(data, ls):
    """
    This method takes as input a dataset, its laplacian scores for all features
    and applies distance based entropy feature selection in order to identify
    the best subset of features in the laplacian sense.
    """
    orderedFeatures = np.argsort(ls)
    scores = {}
    for i in range (2,len(ls)):
        selectedFeatures = orderedFeatures[:i]
        selectedFeaturesDataset = data[:, selectedFeatures]
        d =sklearn.metrics.pairwise_distances(selectedFeaturesDataset, metric = 'euclidean' )
        beta =10
        mu = 0.5

        d = preprocessing.MinMaxScaler().fit_transform(d)
        e = np.vectorize(distanceEntropy)(d) 
        e = preprocessing.MinMaxScaler().fit_transform(e)
        totalEntropy= np.sum(e)
        scores[i] = totalEntropy
    bestFeatures = orderedFeatures[:list(scores.keys())[np.argmin(scores.values())]]
    return bestFeatures

In [315]:
LaplacianScore(np.array(dff_ls[0:1000]))

array([0.03844938, 0.03844938, 0.61560574, 0.02792343, 0.69022319,
       0.02819194, 0.68890972, 0.01365991, 0.03629535, 0.02688746,
       0.03070909, 0.76026652, 0.57360563, 0.02234513, 0.64215162,
       0.37932256])

In [30]:
dff_ls

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,flight,air_time,distance,hour,minute,carrier_label,tailnum_label,origin_label,dest_label
0,2014,1,1,1.0,96.0,235.0,70.0,145,194.0,1542,0.0,1.0,1,1284,0,1
1,2014,1,1,4.0,-6.0,738.0,-23.0,1830,252.0,2279,0.0,4.0,8,167,1,12
2,2014,1,1,8.0,13.0,548.0,-4.0,1609,201.0,1825,0.0,8.0,7,586,0,29
3,2014,1,1,28.0,-2.0,800.0,-23.0,466,251.0,2282,0.0,28.0,8,1426,0,12
4,2014,1,1,34.0,44.0,325.0,43.0,121,201.0,1448,0.0,34.0,1,2275,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162041,2014,9,30,2344.0,-1.0,527.0,-5.0,1215,203.0,1721,23.0,44.0,7,426,1,51
162042,2014,9,30,2347.0,-3.0,229.0,-2.0,121,209.0,1448,23.0,47.0,1,2279,1,1
162043,2014,9,30,2349.0,-1.0,514.0,-19.0,660,188.0,1660,23.0,49.0,1,990,1,17
162044,2014,9,30,2357.0,-2.0,545.0,-20.0,479,210.0,1825,23.0,57.0,7,1167,0,29


In [29]:
dff_ls = dff
dff_ls = dff_ls.drop(columns = cat_col)

In [314]:
np.array(dff_ls[0:1000])

array([[2.014e+03, 1.000e+00, 1.000e+00, ..., 1.284e+03, 0.000e+00,
        1.000e+00],
       [2.014e+03, 1.000e+00, 1.000e+00, ..., 1.670e+02, 1.000e+00,
        1.200e+01],
       [2.014e+03, 1.000e+00, 1.000e+00, ..., 5.860e+02, 0.000e+00,
        2.900e+01],
       ...,
       [2.014e+03, 1.000e+00, 3.000e+00, ..., 6.900e+02, 1.000e+00,
        6.500e+01],
       [2.014e+03, 1.000e+00, 3.000e+00, ..., 2.455e+03, 1.000e+00,
        6.400e+01],
       [2.014e+03, 1.000e+00, 3.000e+00, ..., 1.638e+03, 0.000e+00,
        3.800e+01]])

In [306]:
from sklearn.cluster import DBSCAN, KMeans
from scipy.cluster.hierarchy import dendrogram, linkage ,fcluster
from collections import Counter
from sklearn.datasets.samples_generator import make_blobs
from sklearn import preprocessing
from sklearn.metrics.cluster import adjusted_rand_score
import itertools
import random
from scipy.spatial import distance as ssd
from tqdm import tqdm
import scipy
import scipy.sparse 
from scipy.stats import pearsonr
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph 
from scipy.sparse.linalg import expm
from scipy.linalg import solve_banded
from scipy.spatial.distance import pdist
import scipy.spatial.distance
import math
from sklearn.metrics.pairwise import euclidean_distances
import sklearn
import os.path

def generateData(n_clusters=3,
                 n_samples=20,
                 n_definingfeatures=2,
                 n_shuffle_defining_features=0,
                 n_uniform_features=0,
                 n_normal_features=0):
    """
    This method generates a synthetic dataset with the following properties:
    - n_clusters represents the number of classes, n_samples the number of dimensions. 
    - n_definingfeatures represents the number of features allowing to separate the data and retrieve the 
    generated n_clusters. The defining features are generated with a gaussian distribution (make_blobs)
    - n_shuffle_defining_features shuffles per class the clusters with uniform data, thus creating columns
    When 0 the clusters will have a gaussian distribution across all features
    - n_uniform_features governs the number of non-discriminative columns with an uniform distribution
    - n_uniform_features governs the number of non-discriminative columns with a gaussian distribution
    """
    data_x, data_y = make_blobs(n_samples=n_samples, centers=n_clusters, random_state=42, n_features=n_definingfeatures, 
                               cluster_std=0.05, center_box=(0, 1))

    n_non_shuffle_dims= np.arange(0,n_definingfeatures-n_shuffle_defining_features)

    non_shuffled_data = data_x[:, n_non_shuffle_dims ]
    shuffle_data = data_x[:, len(n_non_shuffle_dims): ]

    uniform_shuffle = np.random.uniform(low=0, high=1, size=(n_samples, n_shuffle_defining_features,))
    # shuffle the dimensions for each cluster
    dims = np.repeat(np.arange(n_shuffle_defining_features *2).reshape(1,-1),n_clusters, axis=0)
    
    for i in range(n_clusters):
        np.random.shuffle(dims[i])
    print(dims)
    cluster_shuffled_features = np.stack([dims[i] for i in data_y])
    mixed_data_to_shuffle =np.hstack([shuffle_data, uniform_shuffle])

    mixed_shuffled_data = np.zeros_like(mixed_data_to_shuffle)
    for i in range(len(mixed_data_to_shuffle)):
        mixed_shuffled_data[i] = mixed_data_to_shuffle[i, :][cluster_shuffled_features[i]]

    uniform_features = np.random.uniform(low=0.1, high=0.9, size=(n_samples, n_uniform_features,))
    normal_features = np.random.normal(size=(n_samples, n_normal_features,))


    data = np.concatenate([non_shuffled_data, mixed_shuffled_data, uniform_features,normal_features], axis=1)
    print(f'discriminative non_shuffled_data {non_shuffled_data.shape}, disc mixed_shuffled_data {mixed_shuffled_data.shape} ' +
         f'uniform_features {uniform_features.shape}, normal_features {normal_features.shape}')
    data = preprocessing.MinMaxScaler().fit_transform(data)
    return data, data_y

n_clusters = 4
n_samples=200
n_definingfeatures=5
n_shuffle_defining_features=3
n_uniform_features=1
n_normal_features=1
minClusterSize = 10 # parameter used for pruning the tree

data, truth = generateData(n_clusters=n_clusters,
                 n_samples=n_samples,
                 n_definingfeatures=n_definingfeatures,
                 n_shuffle_defining_features=n_shuffle_defining_features,
                 n_uniform_features=n_uniform_features,
                 n_normal_features=n_normal_features)
N, M = data.shape
print(Counter(truth), data.shape)

[[5 4 0 3 2 1]
 [0 1 4 3 2 5]
 [5 3 2 4 0 1]
 [4 2 0 1 3 5]]
discriminative non_shuffled_data (200, 2), disc mixed_shuffled_data (200, 6) uniform_features (200, 1), normal_features (200, 1)
Counter({2: 50, 0: 50, 1: 50, 3: 50}) (200, 10)
